In [1]:
import pymongo
import json
import datetime

myclient = pymongo.MongoClient("mongodb://localhost:27017")

mydb = myclient["mydatabase"]
print(myclient.list_database_names())

['admin', 'config', 'local']


In [2]:
mydb["species"].drop()
mydb["events"].drop()


In [3]:
# Insert species data
mycol = mydb["species"]
f = open('species.json')
data = json.load(f)
x = mycol.insert_many(data)

In [4]:
for x in mydb["species"].find():
  print(x)

{'_id': 'Sus Scrofa', 'commonName': 'Wild Pig', 'type': 'Mammal', 'status': 'Invasive', 'diet': 'Omnivore'}
{'_id': 'Cervus Unicolour', 'commonName': 'Sambar Deer', 'type': 'Mammal', 'status': 'Invasive', 'diet': 'Herbavore'}
{'_id': 'Aegotheles Cristatus', 'commonName': 'Australian Owlet-nightjar', 'type': 'Bird', 'status': 'Least Concern', 'diet': 'Carnivore'}
{'_id': 'Alauda Arvensis', 'commonName': 'European Skylark', 'type': 'Bird', 'status': 'Least Concern', 'diet': 'Omnivore'}
{'_id': 'Caligavis Chrysops', 'commonName': 'Yellow-faced Honeyeater', 'type': 'Bird', 'status': 'Least Concern', 'diet': 'Omnivore'}
{'_id': 'Capra Hircus', 'commonName': 'Feral Goat', 'type': 'Mammal', 'status': 'Invasive', 'diet': 'Herbavore'}
{'_id': 'Colluricincla Harmonica', 'commonName': 'Grey Shrikethrush', 'type': 'Bird', 'status': 'Least Concern', 'diet': 'Omnivore'}
{'_id': 'Corvus Coronoides', 'commonName': 'Australian Raven', 'type': 'Bird', 'status': 'Least Concern', 'diet': 'Omnivore'}
{'_id

In [5]:
# insert some detection events:
mycol = mydb["events"]
f = open('events.json')
data = json.load(f)


'''for item in data['timestamp']:
    item['iscategorical'] = item['iscategorical'].replace('$home', item['id'])
d = '''
for item in data:
    s = datetime.datetime.strptime(item['timestamp'], "%Y-%m-%dT%H:%M:%S.000Z")
    print(s)
    item['timestamp'] = s

for item in data:
    print(item)

2023-03-22 13:45:12
2023-03-22 13:55:40
2023-03-22 14:55:44
2023-03-22 15:20:01
2023-03-23 05:55:53
2023-03-24 13:20:13
{'timestamp': datetime.datetime(2023, 3, 22, 13, 45, 12), 'sensorId': 2, 'species': 'Sus Scrofa', 'microphoneLLA': [-33.1101, 150.0567, 23], 'animalEstLLA': [-33.1105, 150.0569, 23], 'animalTrueLLA': [-33.1106, 150.057, 23], 'animalLLAUncertainty': 10, 'audioClip': 'some audio_base64 data', 'confidence': 99.4}
{'timestamp': datetime.datetime(2023, 3, 22, 13, 55, 40), 'sensorId': 2, 'species': 'Cervus Unicolour', 'microphoneLLA': [-33.1101, 150.0567, 23], 'animalEstLLA': [-33.1543, 150.2007, 23], 'animalTrueLLA': [-33.1555, 150.2009, 23], 'animalLLAUncertainty': 10, 'audioClip': 'some audio_base64 data', 'confidence': 98.5}
{'timestamp': datetime.datetime(2023, 3, 22, 14, 55, 44), 'sensorId': 1, 'species': 'Aegotheles Cristatus', 'microphoneLLA': [-33.1121, 150.0577, 24], 'animalEstLLA': [-33.1543, 150.2007, 23], 'animalTrueLLA': [-33.1555, 150.2009, 23], 'animalLLAUnc

In [6]:
x = mycol.insert_many(data)

In [7]:
for x in mydb["events"].find():
  print(x)

{'_id': ObjectId('642e36a12880156c39ff50d7'), 'timestamp': datetime.datetime(2023, 3, 22, 13, 45, 12), 'sensorId': 2, 'species': 'Sus Scrofa', 'microphoneLLA': [-33.1101, 150.0567, 23], 'animalEstLLA': [-33.1105, 150.0569, 23], 'animalTrueLLA': [-33.1106, 150.057, 23], 'animalLLAUncertainty': 10, 'audioClip': 'some audio_base64 data', 'confidence': 99.4}
{'_id': ObjectId('642e36a12880156c39ff50d8'), 'timestamp': datetime.datetime(2023, 3, 22, 13, 55, 40), 'sensorId': 2, 'species': 'Cervus Unicolour', 'microphoneLLA': [-33.1101, 150.0567, 23], 'animalEstLLA': [-33.1543, 150.2007, 23], 'animalTrueLLA': [-33.1555, 150.2009, 23], 'animalLLAUncertainty': 10, 'audioClip': 'some audio_base64 data', 'confidence': 98.5}
{'_id': ObjectId('642e36a12880156c39ff50d9'), 'timestamp': datetime.datetime(2023, 3, 22, 14, 55, 44), 'sensorId': 1, 'species': 'Aegotheles Cristatus', 'microphoneLLA': [-33.1121, 150.0577, 24], 'animalEstLLA': [-33.1543, 150.2007, 23], 'animalTrueLLA': [-33.1555, 150.2009, 23]

In [8]:

start = datetime.datetime(2023, 3, 22, 13, 44)
end = datetime.datetime(2023, 3, 23, 13, 44)
aggregate = [
        {
            '$lookup': {
                'from': 'species',
                'localField': 'species',
                'foreignField': '_id',
                'as': 'info'
            }
        },
        {
            '$match':{'timestamp': {'$lt' : end, '$gte' : start }}
        }

    ]

In [9]:
result = mydb["events"].aggregate(aggregate)

In [10]:

for i in result:
    print(i)

{'_id': ObjectId('642e36a12880156c39ff50d7'), 'timestamp': datetime.datetime(2023, 3, 22, 13, 45, 12), 'sensorId': 2, 'species': 'Sus Scrofa', 'microphoneLLA': [-33.1101, 150.0567, 23], 'animalEstLLA': [-33.1105, 150.0569, 23], 'animalTrueLLA': [-33.1106, 150.057, 23], 'animalLLAUncertainty': 10, 'audioClip': 'some audio_base64 data', 'confidence': 99.4, 'info': [{'_id': 'Sus Scrofa', 'commonName': 'Wild Pig', 'type': 'Mammal', 'status': 'Invasive', 'diet': 'Omnivore'}]}
{'_id': ObjectId('642e36a12880156c39ff50d8'), 'timestamp': datetime.datetime(2023, 3, 22, 13, 55, 40), 'sensorId': 2, 'species': 'Cervus Unicolour', 'microphoneLLA': [-33.1101, 150.0567, 23], 'animalEstLLA': [-33.1543, 150.2007, 23], 'animalTrueLLA': [-33.1555, 150.2009, 23], 'animalLLAUncertainty': 10, 'audioClip': 'some audio_base64 data', 'confidence': 98.5, 'info': [{'_id': 'Cervus Unicolour', 'commonName': 'Sambar Deer', 'type': 'Mammal', 'status': 'Invasive', 'diet': 'Herbavore'}]}
{'_id': ObjectId('642e36a128801